# Decision Tree and Random Forest (RU, EN)

В данном ноутбуке я разбираю, как работают DecisionTree и RandomForestClassifier из библиотеки sklearn для обучения классификатора в задаче [Airline Passenger Satisfaction](https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction) и показываю способ нахождения наиболее важных фичей в определении удовлетворённости пассажира авиакомпаниии.

In this notebook, I'm looking at how DecisionTree and RandomForestClassifier from the sklearn library work to train the classifier in the [Airline Passenger Satisfaction](https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction) problem and show a way to determine the most important features in determining passenger satisfaction with the airline.

**ИТОГОВЫЙ РЕЗУЛЬТАТ (FINAL RESULT): accuracy=0.96, ROC AUC=0.99**

![](https://img2.goodfon.ru/original/1999x1333/8/4c/park-les-derevya-priroda.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Загрузка данных (Data loading)

In [ ]:
SEED = 15
train = pd.read_csv('../input/airline-passenger-satisfaction/train.csv')
test = pd.read_csv('../input/airline-passenger-satisfaction/test.csv')

In [ ]:
train.head()

# Импорт необходимыx библиотек (Importing the necessary libraries)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import tree

Подробнее о работе этих инструментов мозно прочитать, перейдя по этим ссылкам.

You can read more about how these tools work by clicking on these links.

* [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?highlight=decisiontreeclassifier#sklearn.tree.DecisionTreeClassifier)
* [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train_test_split#sklearn.model_selection.train_test_split)
* [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=random%20forest#sklearn.ensemble.RandomForestClassifier)
* [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV)
* [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html?highlight=labelencoder#sklearn.preprocessing.LabelEncoder)
* [tree](https://scikit-learn.org/stable/modules/classes.html?highlight=tree#module-sklearn.tree)

# Подготовка данных для обучения (Preparing data for training)

Функция для очистки датасета от неопределённых значний 

In [ ]:
#clears the dataset of undefined values
def clean_dataset(data):
    assert isinstance(data, pd.DataFrame), "df needs to be a pd.DataFrame"
    data.dropna(inplace=True)
    indices_to_keep = ~data.isin([np.nan, np.inf, -np.inf]).any(1)

Функции для находения категориальных переменных и их замены

In [ ]:
def find_cat(data, max_count_unique=5):
    for name in data.columns:
        s = ''
        s += name
        if type(data[name][0]) == str:
            s += ' is string, '
        if data[name].nunique() <= max_count_unique:
            s += ' few unique'
        if s != name:
            print(s, data[name].unique())

#replacing categorical variables
def encoding_cat(data, max_count_unique=5, msg=True):
    for name in data.columns:
        if type(data[name][0]) == str and data[name].nunique() <= max_count_unique:
            le = LabelEncoder()
            le.fit(data[name])
            data[name] = le.transform(data[name])
    if msg:
        print('Encoding done!')
            

In [ ]:
clean_dataset(train)
find_cat(train)
encoding_cat(train)

In [ ]:
train.info()

Делаем финальные штрихи в подготовке данных.
Подправляем категориальную числовую переменную класса пассажира в соответствии со смыслом. 
Удаляем ненужные строки.

In [ ]:
train.Class = train.Class.replace({0: 3}) 
#Correcting Class variable in accordance with the meaning Eco -> Eco Plus -> Business
train['Arrival Delay in Minutes'].astype('int')
y = train.satisfaction
X = train.drop(['Unnamed: 0', 'id', 'satisfaction'], axis=1)
X.head()

Разделяем обуающую выборку (Splitting the training selection)

![](https://lh6.googleusercontent.com/uuKkYCYun1Ky6C7_GwEtv0gNdaoHyx0WTXiM8jvGOQqGx75gIRVhx1to7OapyGDbOsmKyAl9Eyi5RC-atbk6AXukkA7UBXA-NutKcAdHaTGDsWSzNGyBaCBMvVu1HLuU1Wm6TxWb)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Обучение решающего дерева (Training DecisionTree)

Создаём классификатор и тренируем его (Creating a classifier and training it)

In [ ]:
decision_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth = 10, random_state=SEED)
decision_tree.fit(X_train, y_train)

Формула подсчёта энтропии (formula for calculating entropy criterion)

![](https://i.stack.imgur.com/r5exj.jpg)

Визуализации дерева решений (DecisionTree Visualisation)

In [ ]:
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw
import graphviz  
from sklearn.tree import export_graphviz

# Export our trained model as a .dot file
with open("tree1.dot", 'w') as f:
     f = export_graphviz(decision_tree, out_file=f, max_depth = 4,
                         impurity = True, feature_names = X_train.columns,
                         rounded = True, filled= True )
#Convert .dot to .png to allow display in web notebook
check_call(['dot','-Tpng','tree1.dot','-o','tree.png'])
# Annotating chart with PIL
img = Image.open("tree.png")
draw = ImageDraw.Draw(img)
img.save('sample-out.png')
PImage("sample-out.png")

**Итоговая точность классификатора (final accuracy of the classifier):**

In [ ]:
decision_tree.score(X_val, y_val)

# Обучение случайного леса

![](https://i.ytimg.com/vi/goPiwckWE9M/maxresdefault.jpg)

Попробуем улучшить классификацию и увеличим число деревьев. Каждое дерево независмо друг от друга будет производить классификацию. А после этого деревья (лес) примут общее решение в результате голосования, тем самым, подстраховывая друг друга от ошибок.

Let's try to improve the classification and increase the number of trees. Each tree will classify independently of each other. And after that, the trees (forest) will make a common decision as a result of voting, thereby protecting each other from mistakes

In [ ]:
def search_param(model, param, X_train, y_train, X_val, y_val, area=range(1, 11), msg=True, plot=True, seed=None):
    import matplotlib.pyplot as plt
    import time
    score_list = []
    if msg:
        print('#     accuracy  time')
    for i in area:
        start = time.time()
        rfc = eval(model + '(' + param + '=' + str(i) + ', random_state=' + str(seed) + ')')
        rfc.fit(X_train, y_train)
        s = rfc.score(X_val, y_val)
        end = time.time()
        score_list.append(s)
        if msg:
            print("%-3d %10f  %7f" % (i, s, end - start))
    if plot:
        plt.plot(list(area), score_list)
    return list(area)[score_list.index(max(score_list))]

Производим поиск оптимальных параметров случайного леса (search optimal parameters of a RandomForest)

**n_estimators** 

Количество деревьев в лесу.

The number of trees in the forest.

In [ ]:
search_param('RandomForestClassifier', 'n_estimators', X_train, y_train, X_val, y_val, area=range(1, 51), seed=SEED)

**max_depth**

Максимальная глубина дерева. Если нет, то узлы расширяются до тех пор, пока все листья не станут чистыми или пока все листья не будут содержать меньше образцов min_samples_split.

The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

In [ ]:
search_param('RandomForestClassifier', 'max_depth', X_train, y_train, X_val, y_val, range(1, 25), seed=SEED)

**min_samples_split**

Минимальное количество выборок, необходимых для разделения внутреннего узла:
* Если int, то рассмотрим min_samples_split как минимальное число.
* Если float, то min_samples_split-это дробь, а ceil(min_samples_split * n_samples) - минимальное количество выборок для каждого разделения.

The minimum number of samples required to split an internal node:
* If int, then consider min_samples_split as the minimum number.
* If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.

In [ ]:
search_param('RandomForestClassifier', 'min_samples_split', X_train, y_train, X_val, y_val, range(2, 10), seed=SEED)

**min_samples_leaf**

Минимальное количество примеров, требуемое для нахождения в листовом узле. Точка разделения на любой глубине будет рассматриваться только в том случае, если она оставляет по крайней мере обучающие выборки min_samples_leaf в каждой из левых и правых ветвей.
* Если int, то считайте min_samples_leaf минимальным числом.
* Если float, то min_samples_leaf-это дробь, а ceil(min_samples_leaf * n_samples) - минимальное количество выборок для каждого узла.

The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches.
* If int, then consider min_samples_leaf as the minimum number.
* If float, then min_samples_leaf is a fraction and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node.

In [ ]:
search_param('RandomForestClassifier', 'min_samples_leaf', X_train, y_train, X_val, y_val, range(1, 10), seed=SEED)

**Финальный отбор лучших алгоритмов (final selection of the best algorithms)**

![](https://cf2.ppt-online.org/files2/slide/v/VXgjNpHShCby6zFJdWZeUocmsLTvaltkQERxfOuqP7/slide-12.jpg)

При отборе использется кросс-валидация. Кросс-валидация или скользящий контроль — процедура эмпирического оценивания обобщающей способности алгоритмов. С помощью кросс-валидации эмулируется наличие тестовой выборки, которая не участвует в обучении, но для которой известны правильные ответы.

Cross-validation is used for selection. Cross-validation or sliding control is a procedure for empirically evaluating the generalizing ability of algorithms. Cross-validation emulates the presence of a test sample that does not participate in training, but for which the correct answers are known.

In [ ]:
rfc = RandomForestClassifier(random_state=SEED)
param = {'n_estimators': [i for i in range(38, 51)], 'max_depth': [i for i in range(20, 25)]}
gscv =  GridSearchCV(rfc, param, cv=3, n_jobs=-1, verbose=1)
gscv.fit(X_train, y_train)

Вывод лучших параметров классификатора (output of the best classifier parameters)

In [ ]:
gscv.best_params_

# Результаты обучения

Визуализация важности фичей в определении удовлетворённости пассажиров

Visualization of the importance of features in determining passenger satisfaction

In [ ]:
best_c = gscv.best_estimator_
imp = pd.DataFrame(best_c.feature_importances_, index=X_train.columns, columns=['importance'])
imp.sort_values('importance').plot(kind='barh', figsize=(12, 8))

In [ ]:
best_c.score(X_val, y_val)

Итоговая проверка алгоритма (final verification of the algorithm)

In [ ]:
clean_dataset(test)
find_cat(test)
encoding_cat(test)
test.Class = test.Class.replace({0: 3})
test['Arrival Delay in Minutes'].astype('int')
y_test = test['satisfaction']
X_test = test.drop(['Unnamed: 0', 'id', 'satisfaction'], axis=1)
best_c.score(X_test, y_test)

Визуализируем метрику ROC AUC (Visualize the ROC AUC metric)

In [ ]:
from sklearn.metrics import roc_auc_score , roc_curve
import matplotlib.pyplot as plt
dtc_proba=best_c.predict_proba(X_test)
dtc_proba=dtc_proba[:,1]
auc=roc_auc_score(y_test, dtc_proba)
print('Random Forest Classifier: ROC AUC=%.3f' % (auc))
lr_fpr, lr_tpr, _ = roc_curve(y_test, dtc_proba)
plt.plot(lr_fpr, lr_tpr, marker='.', label='Random Forest Classifier')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
!rm -rf ./sample-out.png ./tree1.dot ./tree.png

**ИТОГОВЫЙ РЕЗУЛЬТАТ(FINAL RESULT): accuracy=0.96, ROC AUC=0.99**